In [1]:
from collections import Counter
from datetime import date, timedelta, datetime
from dateutil.relativedelta import relativedelta
from itertools import combinations, tee
from matplotlib.ticker import PercentFormatter
import matplotlib.lines as mlines
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import re, csv, pickle, spacy, nltk, os, random, json, time, random, csv, os, datetime, lxml, glob, imutils, operator
import numpy as np
import pandas as pd
import researchpy as rp
import scipy.sparse as ss
import scipy.stats as stats
import seaborn as sns
from htmlparsing_kit import clean_sections
import statsmodels.api as sm
import statsmodels.formula.api as smf
import pgeocode
import geopandas as gpd
from shapely.geometry import Point, Polygon
import matplotlib.pyplot as plt
from folium.plugins import TimeSliderChoropleth
import pandas as pd
import geopandas as gpd
import folium
import branca.colormap as cm
from folium.plugins import HeatMap, MarkerCluster
from pyproj import CRS
from folium import plugins
from colour import Color
from branca.element import Figure
from html2image import Html2Image
from bs4 import BeautifulSoup as bs
from PIL import Image, ImageFilter, ImageEnhance
%matplotlib inline

def create_xticks(dtf):
    qrtrs = []
    qlist = sorted(list(set(dtf['quarter'].tolist())))
    for i,qr in enumerate(qlist):
        qr = str(qr)
        qtr = qr.split('Q')[1]
        year = qr.split('Q')[0]
        if qtr == '1':
            newqr = '\n\n'.join([qtr,year])
        else:
            newqr = qtr
        qrtrs.append(newqr)
    return qrtrs, len(qlist)

def get_address(x):
    x = x.split(":",1)[-1].split(":",1)[-1].split(":",1)[-1].split(":",1)[-1].split(":",1)[-1].split(":",1)[-1].split(":",1)[0].strip()
    if x.endswith("Between"):
        address = x.split("Between",1)[0].strip()
    else:
        address = 'missing'
    return address

def write_json(jsonobj,filename):
    with open(filename,'a') as f:
        f.write(json.dumps(jsonobj)+'\n')
        
def get_addressloc(x,nomi,mainpath):
    if x != "missing":
        try:
            place = nomi.query_postal_code(x)
            pdf = pd.DataFrame(place).T
            pdf['query'] = x
            pdf.to_csv(mainpath + "output/7_addresses.csv",sep='\t',index = False,mode='a')
            locale = place['place_name']
        except:
            locale = "error"
    else:
        locale = "missing"
    return locale

def get_llttcolorrange(tcolors,cdf,metric):
    colorrangedict = {}
    for k,v in tcolors.items():
        subcdf = cdf[cdf['tname']==k]
        winrates = sorted(list(set(subcdf[metric].tolist())))
        try:
            wincolors = [str(x) for x in list(Color(v[0]).range_to(Color(v[1]),len(winrates)))]
            colorrangedict[k] = wincolors
        except:
            continue
    return colorrangedict

def plot_map(cdf_i,metric,tcolortype):
    fig=Figure(width=400,height=400)
    onmap = folium.Map(location=[43.67, -79.42], zoom_start=12, tiles='Stamen Toner', control_scale=True, prefer_canvas=True,min_zoom=6)
    fig.add_child(onmap)
    for n,g in cdf_i.groupby('tname'):
        g = g[g[metric] >=0.5]
        for i in range(0,len(g)):
            folium.Circle(
              location=[g.iloc[i]['lat'], g.iloc[i]['long']],
              popup=g.iloc[i]['name']+": "+str(g.iloc[i]['count']),
              radius=float(g.iloc[i][metric])/float(g.iloc[i]["count"])*300,
              color=g.iloc[i][tcolortype][0],
              fill=False,
            ).add_to(onmap)
    return onmap

def imagedeal(maptype):
    mappath_html = mainpath + "graphs/maps/"+maptype+".html"
    mappath_png = mainpath + "graphs/maps/images/"+maptype+".png"    
    with open(mappath_html,"r") as f:
        soup = bs(f.read(),"html.parser")
    hti.screenshot(html_str=str(soup), save_as= maptype+".png", size=(500, 500))
    im = Image.open(mappath_png)
    print(im.size)
    im = im.filter(ImageFilter.SHARPEN)
    im = ImageEnhance.Contrast(im).enhance(10)
    im.crop((60, 60, 450, 450)).save(mappath_png, quality=1000)

nomi = pgeocode.Nominatim('ca')

mainpath = str(os.getcwd())+'/'
hti = Html2Image(output_path=mainpath + "graphs/maps/images/")

newtopicnames = {
    11:"Rent Changes", 
    10:"Moratorium", 
    12:"Nonpayment",
    13:"Failed Condition",
    9:"Nonpayment",
    5:"Moratorium",
    7:"Nonpayment",
    6:"Nonpayment",
    4:"Nonpayment",
    8:"Agree Terminate",
    14:"Nonpayment",
    2:"Nonpayment",
    0:"Failed Condition",
    3:"Breaking Rules",
    1:"Nonpayment"
}
print('------------------------------------------')
df20 = pd.read_csv(mainpath+"output/6_case_db_202021.csv",sep='\t')
df20['datetime'] = pd.to_datetime(df20['date'])
df20['year'] = df20['datetime'].dt.year
df20['monthyear'] = pd.PeriodIndex(df20.datetime, freq='M')
df20['quarter'] = pd.PeriodIndex(df20.datetime, freq='Q')
df20['numone'] = 1
df20['tname'] = df20['topic'].map(newtopicnames)
df20.fillna(np.nan,inplace=True)
print(df20.shape)
print(df20['applicant'].value_counts())
print(df20['winner'].value_counts())
print(pd.crosstab(df20.winner, df20.applicant))
print('------------------------------------------')

df = pd.read_csv(mainpath+"output/5_model_results.csv",sep='\t')
df['cleansections']=df['sections'].apply(lambda x: clean_sections(x))
df['datetime'] = pd.to_datetime(df['date'])
df['year'] =df['datetime'].dt.year
df['quarter'] = pd.PeriodIndex(df.datetime, freq='Q')
df['monthyear'] = pd.PeriodIndex(df.datetime, freq='M')
df['numone'] = 1
print(df.shape)
df = df[df['year']!=2006]
df.fillna(np.nan,inplace=True)
print(df.shape)
print(df['applicant'].value_counts())
print(df['winner'].value_counts())
print(pd.crosstab(df.winner, df.applicant))
print('------------------------------------------')

traindf = pd.read_csv(mainpath+'output/4_traindataplus.csv',sep='\t')
traindf = traindf[traindf['outcome']!='nottrainingsample']
print(traindf.shape)
print(traindf['applicant'].value_counts())
print(traindf['winner'].value_counts())
print(pd.crosstab(traindf.winner, traindf.applicant))
print('------------------------------------------')

df20['address'] =df20['otext'].apply(lambda x: get_address(x))

df20['pcode'] = df20['address'].apply(lambda x: x.split(' ')[-1].strip().lower()[:3] + " " +x.split(' ')[-1].strip().lower()[-3:])
df20['pcode'] = np.where(df20['pcode']=="mis ing","missing",df20['pcode'])

# df20['place'] = df20['pcode'].apply(lambda x: get_addressloc(x,nomi,mainpath))

print(df20.shape)
pdf = pd.read_csv(mainpath + "output/7_addresses.csv",sep='\t')
pdf = pdf[pdf['postal_code']!= 'postal_code']
pdf = pdf[~pdf['latitude'].isna()]
pdf = pdf[pdf['state_code'] =='ON']
pdf.rename(columns={'query':'pcode'}, inplace=True)
pdf.drop_duplicates(subset=['pcode'], keep="first",inplace=True)
print(pdf.shape)

2022-02-07 11:36:24.553984: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-02-07 11:36:24.554006: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


------------------------------------------
(17744, 37)
l    17271
t      473
Name: applicant, dtype: int64
l    15380
t     2364
Name: winner, dtype: int64
applicant      l    t
winner               
l          15255  125
t           2016  348
------------------------------------------
(36461, 35)
(36460, 35)
l    30427
t     6033
Name: applicant, dtype: int64
l    26136
t    10324
Name: winner, dtype: int64
applicant      l     t
winner                
l          24061  2075
t           6366  3958
------------------------------------------
(1636, 29)
l    1133
t     503
Name: applicant, dtype: int64
l    968
t    668
Name: winner, dtype: int64
applicant    l    t
winner             
l          798  170
t          335  333
------------------------------------------
(17744, 39)
(10063, 13)


In [2]:
# random.sample(df20[df20['topic']==5]['text'].tolist(),1)[0]

firstchangedates =[str(x).split(' ',1)[0] for x in pd.date_range(start='1/01/2020', end='31/12/2020')]
secondchangedates =[str(x).split(' ',1)[0] for x in pd.date_range(start='1/01/2021', end='31/12/2021')]

changedatesdict = {}
changedatesdict.update(dict(zip(firstchangedates,["change1"]*len(firstchangedates))))
changedatesdict.update(dict(zip(secondchangedates,["change2"]*len(secondchangedates))))
df20['changes'] = df20['date'].map(changedatesdict)
df20['changes'].value_counts()

pdf['longitude'] = pdf['longitude'].astype(float)
pdf['latitude'] = pdf['latitude'].astype(float)
pcodes = sorted([x.upper() for x in list(set(pdf['postal_code'].tolist()))])
print(len(pcodes))
pcodesdict = dict(zip(pcodes,list(range(0,len(pcodes)+1))))
pdf2 = pdf[['place_name','postal_code','latitude','longitude','pcode']]
pdf2 = pdf2.copy()
pdf2['pid'] = pdf2['postal_code'].map(pcodesdict)

df20['postal_code'] = df20['pcode'].map(dict(zip(pdf2['pcode'],pdf2['postal_code'])))
print(df20.shape)

changelist = []
for n, g in df20.groupby('changes'):
    for t,tg in g.groupby('tname'):
        for l,lg in tg.groupby('postal_code'):
            ttwinrate = lg[lg['applicant']=='t'].shape[0]
            llapprate = lg[lg['applicant']=='l'].shape[0]
            changelist.append([n,t,l,ttwinrate,llapprate,lg.shape[0]])
cdf = pd.DataFrame(changelist,columns = ['changes','tname','postal_code','ttapps','llapps','count'])
cdf['lat'] = cdf['postal_code'].map(dict(zip(pdf2['postal_code'],pdf2['latitude'])))
cdf['long'] = cdf['postal_code'].map(dict(zip(pdf2['postal_code'],pdf2['longitude'])))
cdf['name'] = cdf['postal_code'].map(dict(zip(pdf2['postal_code'],pdf2['place_name'])))

494
(17744, 41)


In [3]:
cdf['tname'].value_counts()

Nonpayment          873
Breaking Rules      628
Failed Condition    626
Agree Terminate     439
Moratorium          219
Rent Changes         30
Name: tname, dtype: int64

In [4]:
tcolors = {"Nonpayment":["#ffe700","#ffff78"],#yellow
"Failed Condition":["#4e00ff","#a78af7"],#purple
"Agree Terminate":["#006bff","#b2d4f7"], #lightblue
"Breaking Rules":["#00ff4a","#88f7a1"],#green
"Moratorium":["#fa0707","#f74f4f"],#red
"Rent Changes":["#ff00f0","#f197f7"]#pink
}

In [5]:
colorrangedicttt = get_llttcolorrange(tcolors,cdf,"ttapps")
colorrangedictll = get_llttcolorrange(tcolors,cdf,"llapps")

cdf['tcolortt'] = cdf['tname'].map(colorrangedicttt)
cdf['tcolorll'] = cdf['tname'].map(colorrangedictll)

cdf_20 = cdf[cdf['changes']=='change1']
cdf_21 = cdf[cdf['changes']=='change2']

In [6]:
print(cdf['changes'].value_counts())

change2    1694
change1    1121
Name: changes, dtype: int64


In [7]:
ttmap_21 = plot_map(cdf_21,"ttapps","tcolortt")
llmap_21 = plot_map(cdf_21,"llapps","tcolorll")

ttmap_20 = plot_map(cdf_20,"ttapps","tcolortt")
llmap_20 = plot_map(cdf_20,"llapps","tcolorll")

In [12]:
ttmap_20.save(mainpath + "graphs/maps/ttmap_20.html")
llmap_20.save(mainpath + "graphs/maps/llmap_20.html")

ttmap_21.save(mainpath + "graphs/maps/ttmap_21.html")
llmap_21.save(mainpath + "graphs/maps/llmap_21.html")

In [1]:
imagedeal("ttmap_21")
imagedeal("llmap_21")
imagedeal("ttmap_20")
imagedeal("llmap_20")